In [1]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
import sys
from pathlib import Path
from datetime import datetime

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T
importlib.reload(T)

here = os.getcwd()
sys.path.append(os.path.join(here, '..'))
import DatasetLoaders
importlib.reload(DatasetLoaders)
from DatasetLoaders import HRSIDSegmentationDataset
import MaskRCNN_model_wrapper
importlib.reload(MaskRCNN_model_wrapper)
# from MaskRCNN_model_wrapper import MaskRCNNWrapper
from MaskRCNN_model_wrapper import do_training, save_model, get_model
import fiftyone_evaluation as fiftyone_evaluation



ModuleNotFoundError: No module named 'fiftyone_evaluation'

In [ ]:
# get path to directory above
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/k3vinli/ENEE439/Capstone')

In [ ]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [ ]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [ ]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [ ]:
session = fo.launch_app()

In [ ]:
def get_transform_lee():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.LeeFilter(5))
    
    return T.Compose(transforms)

In [ ]:
model = get_model(2, True)

In [ ]:
HRSID_torch_train_lee = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_lee())
HRSID_torch_test_lee = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_lee())

In [ ]:
do_training(model, HRSID_torch_train_lee, HRSID_torch_test_lee, num_epochs=45, step_size=15)

Using cuda
Epoch: [0]  [  0/911]  eta: 1:00:17  lr: 0.000010  loss: 4.7421 (4.7421)  loss_classifier: 0.7418 (0.7418)  loss_box_reg: 0.0787 (0.0787)  loss_mask: 1.5302 (1.5302)  loss_objectness: 2.1106 (2.1106)  loss_rpn_box_reg: 0.2808 (0.2808)  time: 3.9713  data: 3.0715  max mem: 4023
Epoch: [0]  [ 10/911]  eta: 0:13:26  lr: 0.000065  loss: 2.6409 (3.0795)  loss_classifier: 0.7125 (0.7171)  loss_box_reg: 0.1776 (0.1551)  loss_mask: 1.5022 (1.5085)  loss_objectness: 0.2470 (0.6203)  loss_rpn_box_reg: 0.0328 (0.0785)  time: 0.8954  data: 0.4384  max mem: 4195
Epoch: [0]  [ 20/911]  eta: 0:10:06  lr: 0.000120  loss: 2.4688 (2.8632)  loss_classifier: 0.6048 (0.6077)  loss_box_reg: 0.1336 (0.1437)  loss_mask: 1.3307 (1.2719)  loss_objectness: 0.1101 (0.7513)  loss_rpn_box_reg: 0.0236 (0.0886)  time: 0.5164  data: 0.1054  max mem: 4264
Epoch: [0]  [ 30/911]  eta: 0:08:48  lr: 0.000175  loss: 1.4049 (2.4242)  loss_classifier: 0.3711 (0.5161)  loss_box_reg: 0.1390 (0.1512)  loss_mask: 0.730

: 

: 

In [ ]:
save_model(model, "maskrcnn_hrsid_pre_lee_e40_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_lee, dataset_test, field_name="predictions_lee")

In [ ]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_lee", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

In [ ]:
results.mAP()

In [ ]:
results.print_report()

In [ ]:
def get_transform_HF_jitter():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.RandomHorizontalFlip())
    transforms.append(T.ScaleJitter((800, 800)))
    
    return T.Compose(transforms)

In [ ]:
model = get_model(2, True)

In [ ]:
HRSID_torch_train_HF_jitter = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_HF_jitter())
HRSID_torch_test_HF_jitter = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_HF_jitter())

In [ ]:
do_training(model, HRSID_torch_train_HF_jitter, HRSID_torch_test_HF_jitter, num_epochs=45, step_size=15)

In [ ]:
save_model(model, "maskrcnn_hrsid_pre_HF_jitter_e45_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_lee, dataset_test, field_name="predictions_HF_jitter")

In [ ]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_HF_jitter", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

In [ ]:
results.mAP()

In [ ]:
results.print_report()